In [1]:
import cv2
import numpy as np
import pandas as pd
import os

from sklearn.cluster import KMeans

from imutils import face_utils
import dlib

In [2]:
# os.environ["OMP_NUM_THREADS"] = '1'
import warnings
warnings.filterwarnings('ignore')

In [3]:
n_colors = 4

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
data_root = './images/dataset__'
seasons = ['spring', 'summer', 'fall', 'winter']

In [14]:
class PaletteCreator:
    def __init__(self, n_colors=n_colors):
        
        self.n_colors = n_colors

        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        self.right_eye = None
        self.left_eye = None
        self.left_cheek = None
        self.right_cheek = None
        self.lips = None
        self.nose = None
        
    def extract_face_part(self, face_part_points):

        (x, y, w, h) = cv2.boundingRect(face_part_points)
        crop = self.img[y:y+h, x:x+w]
        
        # https://www.researchgate.net/publication/262371199_Explicit_image_detection_using_YCbCr_space_color_model_as_skin_detection
        # filter skin only (YCbCr)
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2YCrCb)
        mask = cv2.inRange(crop, np.array([0, 133, 77]), np.array([255, 173, 127]))
        crop = cv2.bitwise_and(crop, crop, mask=mask)
        crop = cv2.cvtColor(crop, cv2.COLOR_YCrCb2BGR)

        crop = crop[~np.all(crop == [0, 135, 0], axis=-1)]
        crop = crop.reshape(((1, crop.shape[0], 3)))
        
        return crop


    def detect_face_part(self):
        face_parts = [[] for _ in range(len(face_utils.FACIAL_LANDMARKS_IDXS))]

        faces = self.detector(cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY), 1)
        
        if len(faces) == 0:
            return 0

        rect = faces[0]

        shape = self.predictor(cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY), rect)
        shape = face_utils.shape_to_np(shape)

        for idx, (_, (i, j)) in enumerate(face_utils.FACIAL_LANDMARKS_IDXS.items()):
            if idx not in [1, 3]:
                face_parts[idx] = shape[i:j]

        self.lips = self.extract_face_part(np.concatenate((shape[48:60], shape[60:68])))
        self.left_cheek = self.extract_face_part(np.concatenate((shape[29:33], shape[4:9])))
        self.right_cheek = self.extract_face_part(np.concatenate((shape[29:33], shape[10:15])))
        self.right_eye = self.extract_face_part(shape[36:42])
        self.left_eye = self.extract_face_part(shape[42:48])
        self.nose = self.extract_face_part(shape[27:36])

        return len(faces)
        
    def create_palette(self, image_path='image.jpg'):
        
        self.img = cv2.imread(image_path)
        
        if self.img is None:
            os.remove(image_path)
            return None
            
        yes_faces = self.detect_face_part()
        
        if not yes_faces:
            os.remove(image_path)
            return None
        
        stacked_images = np.hstack([self.right_eye, self.left_eye, self.lips, self.left_cheek, self.right_cheek, self.nose])
        stacked_images = stacked_images.reshape(-1, 3)
        
        if stacked_images.shape[0] == 0:
            os.remove(image_path)
            return None
            
        kmeans = KMeans(n_clusters=self.n_colors, n_init=10, random_state=42)
        kmeans.fit(stacked_images)

        cluster_centers = kmeans.cluster_centers_
        
        return cluster_centers, self.lips, self.left_cheek, self.right_cheek, self.right_eye, self.left_eye, self.nose

In [15]:
pc = PaletteCreator(n_colors=n_colors)

In [16]:
def calculate_contrast(img):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    hist /= hist.sum()
    mean = np.mean(hist)
    variance = np.mean((hist - mean) ** 2)
    return variance

In [17]:
for season in seasons:
    columns = ['lips_s', 'face_l_var', 'skin_avg_b']
    for i in range(n_colors):
        columns.append(f'R{i}')
        columns.append(f'G{i}')
        columns.append(f'B{i}')
    for i in range(n_colors):
        columns.append(f'H{i}')
        columns.append(f'S{i}')
        columns.append(f'V{i}')
    for i in range(n_colors):
        columns.append(f'L{i}')
        columns.append(f'a{i}')
        columns.append(f'b{i}')
    columns.append('coolwarm')
    columns.append('season')
    this_df = pd.DataFrame(columns=columns)
    for file in os.listdir(os.path.join(data_root, season)):
        full_path = os.path.join(data_root, season, file)
        img = cv2.imread(full_path)
        if img is None:
            print(f'Error reading {full_path}')
            continue
        try:
            palette, lips, left_cheek, right_cheek, right_eye, left_eye, nose = pc.create_palette(full_path)
        except:
            print(f'Error reading {full_path}')
            continue
        
        palette = np.array([palette], np.uint8)
        hsv_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2HSV)
        lab_palette_ = cv2.cvtColor(palette, cv2.COLOR_BGR2LAB)
        
        skin = np.hstack([left_cheek, right_cheek])
        skin = skin.reshape(-1, 3)
        kmeans = KMeans(n_clusters=n_colors, n_init=10, random_state=42)
        kmeans.fit(skin)
        skin_centers = kmeans.cluster_centers_
        skin_centers = np.array([skin_centers], np.uint8)
        lab_palette = cv2.cvtColor(skin_centers, cv2.COLOR_BGR2LAB)
        mean_lab_skin = np.mean(lab_palette, axis=1)[0]
        
        kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
        lips = lips.reshape(-1, 3)
        kmeans.fit(lips)
        lips_centers = kmeans.cluster_centers_
        lips_centers = np.array([lips_centers], np.uint8)
        lab_palette = cv2.cvtColor(lips_centers, cv2.COLOR_BGR2LAB)
        mean_lab_lips = np.mean(lab_palette, axis=1)[0]
        
        h, w = img.shape[:2]
        try:
            face = detector(img)[0]
        except:
            continue
        face = img[max(0, face.top()):min(face.bottom(), h), max(0, face.left()):min(face.right(), w)]
        face_l_var = calculate_contrast(face)
        
        tmp = np.array([mean_lab_lips[1], face_l_var, mean_lab_skin[2]])
        row = np.concatenate((tmp, skin_centers[0].reshape(-1), hsv_palette[0].reshape(-1), lab_palette_[0].reshape(-1))).tolist()
        row.append('warm' if season == 'spring' or season == 'fall' else 'cool')
        row.append(season)
        this_df.loc[len(this_df)] = row
    print(f'{season}: {len(this_df)} rows')
    this_df.to_csv(f'images/dataset__/{n_colors}_{season}.csv', index=False)

Error reading ./images/dataset__\spring\Screenshot＿20201220－021100＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021239＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021339＿Chrome.jpg
Error reading ./images/dataset__\spring\Screenshot＿20201220－021427＿Chrome.jpg
Error reading ./images/dataset__\spring\강남퍼스널컬러컬러연_봄웜톤연예인9.jpg
Error reading ./images/dataset__\spring\다운로드_(39).jpg
Error reading ./images/dataset__\spring\봄웜톤11.jpg
Error reading ./images/dataset__\spring\아이유_고채도.png
Error reading ./images/dataset__\spring\아이유_노랑.png
Error reading ./images/dataset__\spring\아이유_명도.png
Error reading ./images/dataset__\spring\아이유_저명도3.png
Error reading ./images/dataset__\spring\아이유레드.png
Error reading ./images/dataset__\spring\아이유쿨.png
Error reading ./images/dataset__\spring\첫페이지.png
spring: 43 rows
Error reading ./images/dataset__\summer\1200px-180320_이유비.jpg
Error reading ./images/dataset__\summer\20220302＿044135＿880.jpg
Error reading ./images/d

In [5]:
df = pd.concat([pd.read_csv(f'images/dataset__/{n_colors}_{season}.csv') for season in seasons])
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv(f'images/dataset__/{n_colors}_shuffled.csv', index=False)